In [226]:
import pandas as pd
import re

def dms_to_decimal(dms_str):
    """
    Converts ISO 6709 format: +DDMMSS.SS or -DDDMMSS.SS to decimal degrees.
    """
    sign = -1 if dms_str.startswith('-') else 1
    dms_str = dms_str.lstrip('+-')
    #print(len(dms_str))
    if len(dms_str) == 9:  # Latitude: DDMMSS.SS
        deg = int(dms_str[:2])
        minutes = int(dms_str[2:4])
        seconds = float(dms_str[4:])
       # print(seconds)
        #print('latitude')
        
    elif len(dms_str) == 10:  # Longitude: DDDMMSS.SS
        deg = int(dms_str[:3])
        minutes = int(dms_str[3:5])
        seconds = float(dms_str[5:])
       # print(seconds)
       # print('Longitude')
    else:
       # print(len(dms_str))
        return None

    return sign * (deg + minutes / 60 + seconds / 3600)
    #print(sign * (deg + minutes / 60 + seconds / 3600))



In [242]:

def parse_log_file(filepath):
    with open(filepath, 'r', encoding='latin1') as f:
        lines = f.readlines()
        data = {
                'station': None,
                'latitude': None,
                'longitude': None,
                'elevation_m': None,
                'is_greenland': False
            }
    

    for line in lines:
        if 'Four Character ID' in line:
            match = re.search(r':\s*(\w{4})', line)
            if match:
                data['station'] = match.group(1)[:4]
                #print(data['station'])
        if 'Nine Character ID' in line:
            match = re.search(r':\s*(\w{9})', line)
            if match:
                data['station'] = match.group(1)[:4]
                #print(data['station'])
        if 'Latitude (N is +)' in line:
            match = re.search(r':\s*([+-]?\d+(?:\.\d+)?)', line)
            if match:
                data['latitude'] = dms_to_decimal(match.group(1))
                #print(match.group(1))
        if 'Longitude (E is +)' in line:
            match = re.search(r':\s*([+-]?\d+(?:\.\d+)?)', line)
            if match:
                data['longitude'] = dms_to_decimal(match.group(1))
                #print(match.group(1))
        if 'Elevation (m,ellips.)' in line:
            match = re.search(r':\s*([\d.]+)', line)
            if match:
                data['elevation_m'] = float(match.group(1))
        if 'Country' in line:
            match = re.search('Greenland', line)
            if match:
                data['is_greenland'] = True
        if 'Country or Region' in line:
            match = re.search('GRL', line)
            if match:
                data['is_greenland'] = True

    return data


In [248]:
from glob import glob
import os
import pandas as pd

folder = '/Users/dlugardo/Desktop/data/SITELOGS/'  # Your folder
log_files = glob(os.path.join(folder, '*.log'))
#print(log_files)

all_data = [parse_log_file(f) for f in log_files]
greenland_data = [entry for entry in all_data if entry.get('is_greenland') is True]

df_all = pd.DataFrame(greenland_data)

df_all.to_csv('/Users/dlugardo/Documents/GitHub/signal-processing-enu/GreenlandStations.csv') 
#print(df_all.head())